# Analyzing Scenario Results

In [45]:
from datetime import datetime, timedelta
from src.configs import (
    ADDRESSES,
    TOKEN_DTOs,
    get_scenario_config,
    get_price_config,
    CRVUSD_DTO,
)
import pickle
import pandas as pd
import json
import os
import numpy as np
from crvusdsim.pool import get  # type: ignore
from copy import deepcopy
from src.sim.scenario import Scenario

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
def get_results():
    """
    Get all the result objects in the result directory.
    """
    results = []
    dir_ = f"../results/"
    scenario_dirs = [os.path.join(dir_, f) for f in os.listdir(dir_)]
    for scenario_dir in scenario_dirs:
        results_per_scenario = []
        files = [os.path.join(scenario_dir, f) for f in os.listdir(scenario_dir)]
        for file in files:
            with open(file, "rb") as f:
                result = pickle.load(f)
            results_per_scenario.append(result)
        results.extend(results_per_scenario)
    return results


def compare(results) -> pd.DataFrame:
    stats = []
    for i, result in enumerate(results):
        starting_debt = (
            sum([datum.df["Total Debt"].iloc[0] for datum in result.data])
            / len(result.data)
            / 1e6
        )
        starting_crvusd_liquidity = (
            sum([datum.df["Total crvUSD Liquidity"].iloc[0] for datum in result.data])
            / len(result.data)
            / 1e6
        )

        stats_per_run = {
            "Name": result.metadata["template"].name,
            f"Starting Debt (Mns)": starting_debt,
            f"Starting Debt:Liquidity Ratio": starting_debt / starting_crvusd_liquidity,
        }

        for metric in [
            "Bad Debt Pct",
            "Debt Liquidated Pct",
            "Soft Liquidation Loss Pct",
            # "Borrower Loss Pct",
            # "Aggregator Price",
        ]:
            # stats_per_run[f"{metric} Mean"] = result.summary[f"{metric} Max"].mean()
            stats_per_run[f"{metric} Median"] = result.summary[f"{metric} Max"].median()
            # stats_per_run[f"{metric} p90"] = result.summary[f"{metric} Max"].quantile(0.90)
            stats_per_run[f"{metric} p99"] = result.summary[f"{metric} Max"].quantile(
                0.99
            )

        stats.append(stats_per_run)
    df = pd.DataFrame(stats).astype(float, errors="ignore").round(3)
    df.sort_values(["Name"])
    return df

In [83]:
results = get_results()

In [104]:
df = compare(results)
df

,Name,Starting Debt (Mns),Starting Debt:Liquidity Ratio,Bad Debt Pct Median,Bad Debt Pct p99,Debt Liquidated Pct Median,Debt Liquidated Pct p99,Soft Liquidation Loss Pct Median,Soft Liquidation Loss Pct p99
0,Adverse crvUSD Liquidity,113.939,3.134,0.0,0.000,0.000,0.125,0.001,0.005
1,Adverse crvUSD Liquidity,113.937,3.148,0.0,0.000,0.000,0.000,0.001,0.006
2,Adverse crvUSD Liquidity,113.550,3.177,0.0,0.000,0.000,0.043,0.001,0.008
3,Adverse crvUSD Liquidity,113.699,3.183,0.0,0.000,0.000,0.014,0.001,0.007
4,Adverse crvUSD Liquidity,113.334,3.167,0.0,0.002,0.000,0.048,0.001,0.024
5,Adverse vol,114.025,2.428,0.0,0.000,0.057,0.882,0.009,0.069
6,Adverse vol,113.795,2.427,0.0,0.110,0.041,0.947,0.007,0.079
7,Adverse vol,113.442,2.441,0.0,0.186,0.049,1.107,0.008,0.070
8,Adverse vol,113.496,2.455,0.0,0.160,0.068,1.918,0.007,0.069
9,Adverse vol,113.441,2.440,0.0,0.174,0.017,1.230,0.006,0.061


In [105]:
df.groupby(["Name"]).mean().round(3).sort_index()

,Starting Debt (Mns),Starting Debt:Liquidity Ratio,Bad Debt Pct Median,Bad Debt Pct p99,Debt Liquidated Pct Median,Debt Liquidated Pct p99,Soft Liquidation Loss Pct Median,Soft Liquidation Loss Pct p99
Name,,,,,,,,
Adverse Growth,168.184,2.443,0.0,0.000,0.000,0.137,0.001,0.009
Adverse crvUSD Liquidity,113.692,3.162,0.0,0.000,0.000,0.046,0.001,0.010
Adverse drift,113.743,2.438,0.0,0.010,0.000,0.002,0.003,0.025
Adverse vol,113.640,2.438,0.0,0.126,0.046,1.217,0.007,0.070
Baseline,113.745,2.446,0.0,0.004,0.000,0.040,0.001,0.008
Severe crvUSD Liquidity,113.698,5.439,0.0,0.001,0.000,0.145,0.001,0.011
Severe drift,113.717,2.435,0.0,0.011,0.000,0.019,0.007,0.039
Severe vol,113.695,2.441,0.0,0.464,0.219,7.328,0.017,0.202
Severe vol and adverse crvUSD liquidity,113.562,3.174,0.0,0.271,0.246,6.962,0.019,0.183
